**Mushroom Classification: Safe to eat or deadly poison?**
<table align="left">
  <td>
    <a target="_blank" href="https://www.kaggle.com/code/filippodoldi/mushrooms-classification/edit/run/121143299"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>

**Context**

Although this dataset was originally contributed to the UCI Machine Learning repository nearly 30 years ago, mushroom hunting (otherwise known as "shrooming") is enjoying new peaks in popularity. Learn which features spell certain death and which are most palatable in this dataset of mushroom characteristics. And how certain can your model be?

**Content**

This dataset includes descriptions of hypothetical samples corresponding to 23 species of gilled mushrooms in the Agaricus and Lepiota Family Mushroom drawn from The Audubon Society Field Guide to North American Mushrooms (1981). Each species is identified as definitely edible, definitely poisonous, or of unknown edibility and not recommended. This latter class was combined with the poisonous one. The Guide clearly states that there is no simple rule for determining the edibility of a mushroom; no rule like "leaflets three, let it be'' for Poisonous Oak and Ivy.

**Solution**

In this binary classification problem since the poisonus class is the positive class we aim to maximize recall since we want to avoid false negative, i.e. people getting poisoned by eating misclassiefied mushrooms.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preprocessing" data-toc-modified-id="Preprocessing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preprocessing</a></span></li><li><span><a href="#Random-Forest-Classifier" data-toc-modified-id="Random-Forest-Classifier-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Random Forest Classifier</a></span></li><li><span><a href="#Important-features-and-test-results" data-toc-modified-id="Important-features-and-test-results-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Important features and test results</a></span></li></ul></div>

# Preprocessing

In [ ]:
import pandas as pd

df = pd.read_csv("/kaggle/input/mushroom-classification/mushrooms.csv")

Target Information: 

- classes: edible=e, poisonous=p

Attribute Information: 

- cap-shape: bell=b,conical=c,convex=x,flat=f, knobbed=k,sunken=s

- cap-surface: fibrous=f,grooves=g,scaly=y,smooth=s

- cap-color: brown=n,buff=b,cinnamon=c,gray=g,green=r,pink=p,purple=u,red=e,white=w,yellow=y

- bruises: bruises=t,no=f

- odor: almond=a,anise=l,creosote=c,fishy=y,foul=f,musty=m,none=n,pungent=p,spicy=s

- gill-attachment: attached=a,descending=d,free=f,notched=n

- gill-spacing: close=c,crowded=w,distant=d

- gill-size: broad=b,narrow=n

- gill-color: black=k,brown=n,buff=b,chocolate=h,gray=g, green=r,orange=o,pink=p,purple=u,red=e,white=w,yellow=y

- stalk-shape: enlarging=e,tapering=t

- stalk-root: bulbous=b,club=c,cup=u,equal=e,rhizomorphs=z,rooted=r,missing=?

- stalk-surface-above-ring: fibrous=f,scaly=y,silky=k,smooth=s

- stalk-surface-below-ring: fibrous=f,scaly=y,silky=k,smooth=s

- stalk-color-above-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y

- stalk-color-below-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y

- veil-type: partial=p,universal=u

- veil-color: brown=n,orange=o,white=w,yellow=y

- ring-number: none=n,one=o,two=t

- ring-type: cobwebby=c,evanescent=e,flaring=f,large=l,none=n,pendant=p,sheathing=s,zone=z

- spore-print-color: black=k,brown=n,buff=b,chocolate=h,green=r,orange=o,purple=u,white=w,yellow=y

- population: abundant=a,clustered=c,numerous=n,scattered=s,several=v,solitary=y

- habitat: grasses=g,leaves=l,meadows=m,paths=p,urban=u,waste=w,woods=d

All our varibles are nominal, let's check their encoding and if there is some missing values.

In [ ]:
df.info()

We don't have missing values, so we can proced to show some barplots of all the features divided by class (poisonus or edible):

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

colors = np.where(df["class"] == 'p', 'r','g')
for feature in df.columns[1:]:
    fig, axs = plt.subplots(1, 2, sharey = True, figsize=(8, 3))
    
    df[df["class"] == 'p'][feature].value_counts().plot(kind='bar', rot=0, ax = axs[0], color = 'purple')
    axs[0].set_title("Poisonus")
    
    df[df["class"] == 'e'][feature].value_counts().plot(kind='bar', rot=0, ax = axs[1], color = 'green')
    axs[1].set_title("Edible")
    fig.suptitle(feature)
    plt.show()

We can see that a lot of categories are exclusive of one class, for example if we look at odor we notice that only 'n' = none is shared among the two classes; this types of divisions will be very impactful on the result of the final model.

Also we notice that the 'veil-type' column takes just one value 'p' so we discard it.

In [ ]:
df = df.drop(columns = ["veil-type"])

# Random Forest Classifier

In order to speed up the process let's encode the letter as numbers, we will use an ordinal encoder even if we won't care about the numbers' order.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

X = pd.DataFrame(OrdinalEncoder(dtype = int).fit_transform(df), columns = df.columns)

After encoding the poisonus class is assigned to be the positive class.

Now, since we have a lot of categorical variable with very different values we will avoid one-hot encoding and we will use Random forests which can handle very well this type of nominal variables. So let's create the target and split the dataset before training the model:

In [ ]:
from sklearn.model_selection import train_test_split

target = X["class"].copy()
X = X.drop(columns = ["class"]).copy()

rs = 10
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size = 0.3, random_state = rs)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, max_leaf_nodes=16, max_depth = 10, max_features = 10,
                                 n_jobs=-1, random_state=rs)
model.fit(X_train, y_train)

In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np
score = cross_val_score(model, X_train, y_train, cv = 5, scoring = "recall")

print("mean_recall: ", np.mean(score))
print("recall_std: ", np.std(score))

We achieve sensational results even without tuning the hyperparameters. To achieve even better results let's use grid search:

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
 {'n_estimators': [100, 150, 200, 300], 
  'max_features': [10, 13, 16, 19],
  'max_leaf_nodes': [13, 16, 19],
  'max_depth': [8, 10, 12],
 },
 ]

grid_search = GridSearchCV(model, param_grid, cv=5, scoring = 'recall')
grid_search.fit(X_train, y_train)

Let's check the parameters and the score:

In [ ]:
final_model = grid_search.best_estimator_ 

print(grid_search.best_params_ )

In [ ]:
print(grid_search.best_score_ )

We achieved perfection, let's check its standard deviation through cross-validation:

In [ ]:
recall = cross_val_score(final_model, X_train, y_train, cv = 5, scoring = "recall")

print("mean_recall: ", np.mean(recall))
print("recall_std: ", np.std(recall))

Since we have maximized recall, let's check also the precision for completness:

In [ ]:
precision = cross_val_score(final_model, X_train, y_train, cv = 5, scoring = "precision")

print("mean_precision: ", np.mean(precision))
print("precision_std: ", np.std(precision))

This mean also that we have 100% accuracy.

# Important features and test results

Now let's look at the features that are the most meaningful in the prediction:

In [ ]:
feature_importances = final_model.feature_importances_
feature_importances.round(1)

In [ ]:
indexes = feature_importances > 0.05
important_features = X.columns[indexes]

In [ ]:
important_features

Let's see again the barplot of this important features:

In [ ]:
import matplotlib.pyplot as plt
colors = np.where(df["class"] == 'p', 'r','g')
for feature in important_features:
    fig, axs = plt.subplots(1, 2, sharey = True, figsize=(8, 3))
    
    df[df["class"] == 'p'][feature].value_counts().plot(kind='bar', rot=0, ax = axs[0], color = 'purple')
    axs[0].set_title("Poisonus")
    
    df[df["class"] == 'e'][feature].value_counts().plot(kind='bar', rot=0, ax = axs[1], color = 'green')
    axs[1].set_title("Edible")
    fig.suptitle(feature)
    plt.show()

We can see that all these features have values that are well separeted in the two classes.

Finally let's evaluate our system on the test set:

In [ ]:
from sklearn.metrics import recall_score,precision_score, accuracy_score

y_pred = final_model.predict(X_test)
print(recall_score(y_test,y_pred))
print(precision_score(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

**We have a perfect prediction.**